In [ ]:
# code block for imports 
import pandas as pd 
import numpy as np
import sqlite3
from sqlalchemy import create_engine
import os
import matplotlib.pyplot as plt
import pandas as pd
import folium
from folium import plugins
from folium.plugins import HeatMap

In [ ]:
conn = sqlite3.connect('cs506MAPC.db')

## Display first 10 Providers 

In [ ]:
qrystr ="SELECT DISTINCT t1.ProviderName,substr(t1.BlockCode,0,12) as tractNum,t1.MaxAdDown,t1.MaxAdUp, t2.town, t3.Longitude, t3.Latitude FROM FCC_DATA_T t1, censusblocks_t t2, CityLongLat_t t3 WHERE substr(t1.BlockCode,0,12) = t2.blockcode AND t2.town = t3.town ORDER BY t2.town ASC"


#convert query results into dataframe
df_filtered_providers = pd.read_sql_query(qrystr, conn)
print(df_filtered_providers.head(10))

## Non Zero Providers

In [ ]:
qryStr2 = "SELECT DISTINCT t1.ProviderName,substr(t1.BlockCode,0,12) as tractNum,t1.MaxAdDown,t1.MaxAdUp, t2.town, t3.Longitude, t3.Latitude FROM FCC_DATA_T t1, censusblocks_t t2, CityLongLat_t t3 WHERE MaxAdDown != 0.0 AND MaxAdUp != 0.0 and substr(t1.BlockCode,0,12) = t2.blockcode AND t2.town = t3.town ORDER BY t2.town ASC"
df_filtered_non_zero_providers = pd.read_sql_query(qryStr2, conn)
print(df_filtered_non_zero_providers.head(10))


## Zero Providers

In [ ]:
qryStr3 = "SELECT DISTINCT t1.ProviderName,substr(t1.BlockCode,0,12) as tractNum,t1.MaxAdDown,t1.MaxAdUp, t2.town, t3.Longitude, t3.Latitude FROM FCC_DATA_T t1, censusblocks_t t2, CityLongLat_t t3 WHERE MaxAdDown = 0 AND MaxAdUp = 0 and substr(t1.BlockCode,0,12) = t2.blockcode AND t2.town = t3.town ORDER BY t2.town ASC"
df_filtered_zero_providers = pd.read_sql_query(qryStr3, conn)
print(df_filtered_zero_providers.head(10))

## Max Providers Per Area

In [ ]:
town_dict = {}
towns = df_filtered_zero_providers['town']
providers = df_filtered_zero_providers['ProviderName']
df_towns_providers = pd.concat([towns, providers], axis=1)
numpy_arr = df_towns_providers.to_numpy()
for i in range(len(numpy_arr)):
    town_name = numpy_arr[i][0]
    if town_name not in town_dict.keys():
        town_dict[town_name] = [numpy_arr[i][1]]
    elif numpy_arr[i][1] not in town_dict.get(town_name):
        town_dict[town_name].append(numpy_arr[i][1])
    else:
        continue

num_providers_dict = {}
for key in town_dict.keys():
    num_providers_dict[key] = len(town_dict.get(key))

df_num_providers = pd.DataFrame(num_providers_dict.items(), columns=['Town', 'ProviderCount'])
print(df_num_providers)
